# Tutorial - Recommender Systems

The primary goal of this tutorial is to provide a comprehensive understanding of how sparse recommender systems are stored and how to effectively utilize sparse matrices. Additionally, the tutorial aims to guide learners through the process of implementing autoencodeer algorithms for recommenders.

Importing Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

Loading the dataset

In [2]:
ratings = np.load("ROMEGA.npy").astype(int)
ratings

array([[   0,  456,    1],
       [   0,  314,    1],
       [   0,  234,    3],
       ...,
       [ 901, 1054,    3],
       [ 901,  141,    3],
       [ 901, 1423,    1]])

Some statistics of the dataset

In [3]:
users = np.unique(ratings[:,0])
print(f"Users ids: {users[0:10]}")
items = np.unique(ratings[:,1])
print(f"Items ids: {users[0:10]}")

print(f"# of users: {users.shape[0]}")
print(f"# of items: {items.shape[0]}")
print(f"# of observed entries: {ratings.shape[0]}")
print(f"# of possible entries: {users.shape[0]*items.shape[0]}")

print(f"% of observed entries: {ratings.shape[0]/(users.shape[0]*items.shape[0])}")


Users ids: [0 1 2 3 4 5 6 7 8 9]
Items ids: [0 1 2 3 4 5 6 7 8 9]
# of users: 902
# of items: 1500
# of observed entries: 97499
# of possible entries: 1353000
% of observed entries: 0.07206134515890614


The focus of this dataset is on providing movie recommendations to users. Due to privacy concerns, user information is not disclosed. However, the dataset does include information on the first twenty movies that each user has interacted with. This information can be used to build recommender systems that recommends new movies to users based on their past movie preferences.



| Movie Index | Title | IMDB |
| --- | --- | --- |
| 0 | Ace Ventura: Pet Detective    |[Link](https://www.imdb.com/title/tt0109040/?ref_=fn_al_tt_1) |
| 1 | Back to the Future   |[Link](https://www.imdb.com/title/tt0088763/?ref_=nv_sr_srsg_0) |
| 2 | Citizen Kane   |[Link](https://www.imdb.com/title/tt0033467/?ref_=nv_sr_srsg_0) |
| 3 | Dirty Dancing     |[Link](https://www.imdb.com/title/tt0092890/?ref_=nv_sr_srsg_0) |
| 4 | E.T. the Extra-Terrestrial   |[Link](https://www.imdb.com/title/tt0083866/?ref_=nv_sr_srsg_0) |
| 5 | Forrest Gump     |[Link](https://www.imdb.com/title/tt0109830/?ref_=fn_al_tt_1) |
| 6 | Free Willy     |[Link](https://www.imdb.com/title/tt0106965/?ref_=nv_sr_srsg_0) |
| 7 | Ghost    |[Link](https://www.imdb.com/title/tt0099653/?ref_=nv_sr_srsg_3) |
| 8 | GoldenEye     |[Link](https://www.imdb.com/title/tt0113189/?ref_=nv_sr_srsg_0) |
| 9 | Home Alone    |[Link](https://www.imdb.com/title/tt0099785/?ref_=nv_sr_srsg_0) |
| 10 | Independence Day (ID4)    |[Link](https://www.imdb.com/title/tt0116629/?ref_=nv_sr_srsg_2) |
| 11 | Jurassic Park    |[Link](https://www.imdb.com/title/tt0107290/?ref_=nv_sr_srsg_0) |
| 12 | Mortal Kombat   |[Link](https://www.imdb.com/title/tt0113855/?ref_=nv_sr_srsg_3) |
| 13 | Pulp Fiction    |[Link](https://www.imdb.com/title/tt0110912/?ref_=nv_sr_srsg_0) |
| 14 | Terminator 2: Judgment Day   |[Link](https://www.imdb.com/title/tt0103064/?ref_=nv_sr_srsg_0) |
| 15 | The Godfather    |[Link](https://www.imdb.com/title/tt0068646/?ref_=nv_sr_srsg_0) |
| 16 | The Lion King    |[Link](https://www.imdb.com/title/tt0110357/?ref_=nv_sr_srsg_3) |
| 17 | The Mask    |[Link](https://www.imdb.com/title/tt0110475/?ref_=nv_sr_srsg_0) |
| 18 | Toy Story     |[Link](https://www.imdb.com/title/tt0114709/?ref_=nv_sr_srsg_0) |
| 19 | Willy Wonka and the Chocolate Factory    |[Link](https://www.imdb.com/title/tt0067992/?ref_=nv_sr_srsg_0) |



Store in a dense format

In [4]:
m=users.shape[0]
n=items.shape[0]
R_dense = np.repeat(np.nan,m*n).reshape(m,n)
R_dense[ratings[:,0],ratings[:,1]] = ratings[:,2]
R_dense

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [ 5.,  5., nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan,  4., nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])

Store in a sparse format

In [5]:
R_sparse = csr_matrix((ratings[:,2] * 1., (ratings[:,0], ratings[:,1])), shape=(m, n))
R_sparse

<902x1500 sparse matrix of type '<class 'numpy.float64'>'
	with 97499 stored elements in Compressed Sparse Row format>

In [6]:
R_sparse.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [5., 5., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 4., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [7]:
X_train, X_test, y_train, y_test = train_test_split(ratings[:,0:2], ratings[:,2], test_size=0.05, random_state=42)
ratings_train = np.c_[X_train,y_train]
ratings_test = np.c_[X_test,y_test]
print(f"Train : {ratings_train.shape} Test: {ratings_test.shape}")
ratings_train

Train : (92624, 3) Test: (4875, 3)


array([[161, 154,   3],
       [ 16, 739,   3],
       [ 28, 669,   3],
       ...,
       [407, 372,   4],
       [  3, 849,   2],
       [ 40, 433,   3]])

**Shallow Linear Autoencoder**

In recent years, deep learning approaches have shown promising results in collaborative filtering tasks, such as recommendation systems. However, unlike in computer vision, it has been found that using a small number of hidden layers in recommendation systems achieves better accuracy. The Embarrassingly Shallow AutoEncoder (ESAE) is a linear model without hidden layers that uses a binary input vector to indicate user interactions with items and predicts the best items to recommend to the user by reproducing the input as its output, using an autoencoder approach. ESAE is computationally efficient and has been shown to perform well in recommendation tasks.

In [8]:
def EASE(X, lambda_): 
    G = X.T.dot(X).toarray()
    diagIndices = np.diag_indices(G.shape[0])
    G[diagIndices] += lambda_
    P = np.linalg.inv(G)
    B = P / (-np.diag(P))
    B[diagIndices] = 0
    return B

In [9]:
X_sparse = csr_matrix((np.repeat(1.,ratings_train.shape[0]), (ratings_train[:,0], ratings_train[:,1])), shape=(m, n))
X_sparse
print(X_sparse)

  (0, 3)	1.0
  (0, 69)	1.0
  (0, 100)	1.0
  (0, 171)	1.0
  (0, 187)	1.0
  (0, 234)	1.0
  (0, 260)	1.0
  (0, 314)	1.0
  (0, 376)	1.0
  (0, 456)	1.0
  (0, 463)	1.0
  (0, 582)	1.0
  (0, 621)	1.0
  (0, 676)	1.0
  (0, 762)	1.0
  (0, 811)	1.0
  (0, 1066)	1.0
  (0, 1300)	1.0
  (1, 28)	1.0
  (1, 56)	1.0
  (1, 153)	1.0
  (1, 236)	1.0
  (1, 262)	1.0
  (1, 267)	1.0
  (1, 312)	1.0
  :	:
  (900, 694)	1.0
  (900, 848)	1.0
  (900, 944)	1.0
  (900, 1015)	1.0
  (900, 1215)	1.0
  (901, 15)	1.0
  (901, 49)	1.0
  (901, 63)	1.0
  (901, 86)	1.0
  (901, 87)	1.0
  (901, 141)	1.0
  (901, 173)	1.0
  (901, 206)	1.0
  (901, 266)	1.0
  (901, 290)	1.0
  (901, 340)	1.0
  (901, 373)	1.0
  (901, 415)	1.0
  (901, 492)	1.0
  (901, 558)	1.0
  (901, 602)	1.0
  (901, 901)	1.0
  (901, 944)	1.0
  (901, 1054)	1.0
  (901, 1120)	1.0


In [10]:
B=EASE(X = X_sparse, lambda_ = 10)
X_sparse @ B

array([[-8.41364972e-04,  7.51077942e-03,  8.29558994e-02, ...,
        -7.35415684e-03,  2.77143236e-02, -1.68381777e-02],
       [-2.19771278e-02,  9.91313896e-02, -3.66229626e-02, ...,
        -1.14436439e-03, -5.88337931e-02, -8.25180271e-03],
       [ 9.95426483e-01,  9.81666118e-01,  3.53743301e-02, ...,
         8.51025079e-03, -1.08077184e-02,  2.43513051e-03],
       ...,
       [-7.13919663e-02,  9.63373858e-02,  3.43846659e-02, ...,
         8.19262255e-03, -1.22549777e-02, -1.11425388e-02],
       [-9.94964934e-03,  5.20826742e-01,  1.04794523e-02, ...,
         8.86594413e-03, -5.13809290e-03,  1.74329556e-02],
       [ 5.05960263e-02,  2.18667863e-02, -7.49992430e-02, ...,
         5.50144607e-03,  1.87413147e-02,  1.34474618e-02]])

In [11]:
u23 = (X_sparse[23,:] @ B)[0]
print(f"Scores of EASE: {u23}")

Scores of EASE: [ 0.93641534  0.90043209  0.11073922 ...  0.00391504  0.01472662
 -0.00295597]


In [12]:
sequenceScores = np.argsort(-u23)
print(f"Sequence of scores: {sequenceScores}")

Sequence of scores: [500  14 629 ... 420 602 126]


In [13]:
itemsObs23 = ratings_train[ratings_train[:,0]==23,1]
print(f"Items observed by user 23: {itemsObs23}")

sequenceRecommendation = sequenceScores[~np.isin(sequenceScores,itemsObs23)]
print(f"Sequence of recommendation: {sequenceRecommendation}")

print(f"{sequenceScores.shape[0]} {sequenceRecommendation.shape[0]} {itemsObs23.shape[0]}")

Items observed by user 23: [  98  654 1302  599  391  923  994  171  750  536 1421  232  199   76
  891  764  162  427  676  862 1142   49  924  763  916  555  708   13
   20  333  165   34 1261 1057  494   51   55 1005  302  270   27  129
  135  106  211  624 1473  216  262    5   28 1125  690  730   32  293
  752  356  629 1158  323   97  269  408  942  432 1033   89  596  198
  134  224  104  309   72  399   41  844  359  479   62 1166  308  486
  620  820  363  768  552  861  829  227  326  501  762   17  720  243
   75    0  505 1019   74   19  778  883  210  435  166  583  600  651
  694  606  272 1215  118 1100  254  196  687  156  639  203  758  618
  679  757  406  396  565 1000 1476  926  520  813  518 1072 1366 1225
  340  279  745  569   16  653   85  370   35  123  491 1432  959   87
  245  468  464  500  848 1034  636 1415  407 1074  925  252  540  110
  740  831 1167   11 1373 1263  218 1414  241  715  383  814   68   70
  314  847  904  695  467  849  534  968  756  812

In [14]:
def ndcg(scores, true_relevance):
    IDCG = (1/np.log2(np.arange(1,true_relevance.sum()+1)+1)).sum()
    relevant = np.zeros(scores.shape[0])
    relevant[np.argsort(-scores)] = np.arange(1,scores.shape[0]+1) * true_relevance[np.argsort(-scores)]
    DCG = (1/np.log2(relevant[relevant!=0]+1)).sum()
    return DCG / IDCG


u23_test = ratings_test[ratings_test[:,0]==23,1]

true_relevance = np.zeros(u23.shape[0])
true_relevance[u23_test] = 1.

ndcg(u23[~np.isin(sequenceScores,itemsObs23)],true_relevance[~np.isin(sequenceScores,itemsObs23)])



0.299093658523206

In [15]:
from sklearn.metrics import ndcg_score
true_relevance = true_relevance[~np.isin(sequenceScores,itemsObs23)]
true_relevance = true_relevance.reshape(1,true_relevance.shape[0])
scores = u23[~np.isin(sequenceScores,itemsObs23)]
scores = scores.reshape(1,scores.shape[0])
ndcg_score(true_relevance, scores)

0.2990936585232065

# Homework

The homework can be done by groups up to three students. Your task is to design, train and cross-validation a deep autoencoder to achieve the best performance (highest NDCG). Here's what you need to do:

* Chose 3 to 5 of the movies that you have watched (or you think you would watch it) from the table given above and add this information to the end of the data set. For example, if you watched _Home Alone_, add the entry np.array([902,9,1]) to the data set.

* **Data Preparation:** Split the dataset into training, validation, and test sets. Preprocess the data, such as scaling or normalization, to make it suitable for the model.

* **Architecture Selection:** Choose an appropriate architecture for the deep autoencoder. Experiment with different architectures and hyperparameters to find the best model.

* **Model Training:** Train the deep autoencoder using an appropriate optimization algorithm, such as stochastic gradient descent. Monitor the training process using appropriate metrics such as loss and accuracy. Use early stopping and regularization techniques to prevent overfitting.

* **Validation**: Validate the model to find hyperparamters. Note that our aim is to have high NDCG.

* Performance Evaluation: Evaluate the performance of the deep autoencoder on the test set using an appropriate evaluation metric such as NDCG. The group with the highest NDCG score wins the contest.


In [16]:
#your code here